In [2]:
import warnings
warnings.filterwarnings('ignore')

import math
import polars as pl
import sklearn as sklearn
import scipy as scipy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sys
import tensorflow.keras as keras
import tensorflow as tf
import cv2 as opencv
import numpy as np
import yfinance

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, Add, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import plotly
from plotly.graph_objs import *
import plotly.tools as tls
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error, explained_variance_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, ReLU, Add, Dense, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

2024-10-23 22:33:55.908451: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


2024-10-23 22:33:56.913932: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-23 22:33:56.917578: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-10-23 22:33:56.917695: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
def get_yf_hist(symbol_list,startDate,endDate,interval):
  import matplotlib.pyplot as plt
  import datetime
  import yfinance as yf

  # Get historical pricing data
  data = yf.download(symbol_list, startDate, endDate, interval)

  return data


def get_intraday(symbol_list,period,interval):
  intraday_list = []
  intraday_df = pd.DataFrame()
  for sym in symbol_list:
    intraday = yf.download(tickers=sym,
                           period=period,
                           interval=interval)
    intraday['symbol'] = sym
    intraday_df = pd.concat([intraday_df,intraday])

  return intraday_df

def rolling_zscore(data,return_period,window_length):
  log_returns = (np.log(data / data.shift(return_period)))
  zscore = (log_returns - log_returns.rolling(window_length).mean() / log_returns.rolling(window_length).std())
  #results_dict = dict({'log_returns':log_returns})
  results_df = pd.DataFrame(zscore)

  return results_df


  # convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def rmse(y_true,y_pred):
    
    # Ensure both lists have the same length
    if len(y_true) != len(y_pred):
        raise ValueError("The length of actual and predicted values must be the same")
    
    # Calculate the squared differences
    squared_diff = [(yt-yp) ** 2 for yt, yp in zip(y_true,y_pred)]
    
    # Calculate mean of the squared differences
    mean_squared_diff = sum(squared_diff) / len(y_true)
    
    # Return the square root of the mean squared difference
    rmse = math.sqrt(mean_squared_diff)
    
    return rmse

In [108]:
#symbol_list = ['GS','JPM','C','WFC','STT']
#symbol_list = ['^GSPC','AMD','NVDA','TXN']
#ymbol_list = ['^GSPC','SPY','MSFT','AAPL','NVDA','AMZN','LLY','JPM','AMD','XOM','UNH','PG','JNJ','HD','MRK','COST','CVX','NFLX','CRM','WMT','AMD','BAC','PEP','KO','LIN','ADBE','WFC','TMO','DIS','ACN','MCD','CSCO','ORCL','ABT','CAT','QCOM','INTU','AMGN','BKNG','LOW','ISRG','CRM']


symbol_list = ['^GSPC','SPY','TLT','IVV']
startDate = '1900-01-01'
endDate = '2024-10-23'

close_df = get_yf_hist(symbol_list,startDate,endDate,interval='1d')

close_df = close_df['Adj Close']

special_col = '^GSPC'
cols = [special_col] + [col for col in close_df.columns if col != special_col]
close_df = close_df[cols]
close_df

close_df.head()

[*********************100%***********************]  4 of 4 completed


Ticker,^GSPC,IVV,SPY,TLT
Date,,,,
1927-12-30 00:00:00+00:00,17.660000,NaN,NaN,NaN
1928-01-03 00:00:00+00:00,17.760000,NaN,NaN,NaN
1928-01-04 00:00:00+00:00,17.719999,NaN,NaN,NaN
1928-01-05 00:00:00+00:00,17.549999,NaN,NaN,NaN
1928-01-06 00:00:00+00:00,17.660000,NaN,NaN,NaN


In [109]:
close_df = close_df.reset_index()
close_df = close_df.drop(['Date'],axis=1)

In [110]:
df = close_df
df = df.dropna(axis=0,how='any')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5597 entries, 18722 to 24318
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ^GSPC   5597 non-null   float64
 1   IVV     5597 non-null   float64
 2   SPY     5597 non-null   float64
 3   TLT     5597 non-null   float64
dtypes: float64(4)
memory usage: 218.6 KB


In [111]:
#assert not np.isnan(df.values).any(), "Data contains NaN values!"
#assert not np.isinf(df.values).any(), "Data contains infinite values!"


In [112]:
sequence_length = 30  # Number of days to look back
features = df.shape[1]  # Number of features
epochs = 50
batch_size = 16
filters = 64

# Normalize data
scaler = MinMaxScaler()
data = scaler.fit_transform(df)

# Prepare sequences and targets
X, y = [], []
for i in range(sequence_length, len(data)):
    X.append(data[i-sequence_length:i])
    y.append(data[i][0])  # SPY's price as target

X, y = np.array(X), np.array(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# TCN model architecture
def build_tcn(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters, kernel_size=2, dilation_rate=1, padding="causal")(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Stacking TCN blocks with residual connections
    for dilation_rate in [2, 4, 8, 16]:
        previous_x = x
        x = Conv1D(filters, kernel_size=2, dilation_rate=dilation_rate, padding="causal")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Add()([x, previous_x])  # Residual connection
    
    x = Flatten()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    return model

# Compile model
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_tcn(input_shape)
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

# Evaluate model
mse = model.evaluate(X_test, y_test)
print("Test MSE:", mse)

# Predict future SPY prices
predictions = model.predict(X_test)

Epoch 1/50
279/279 [==============================] - 6s 14ms/step - loss: 0.3983 - val_loss: 1.6674
Epoch 2/50
279/279 [==============================] - 3s 11ms/step - loss: 0.0985 - val_loss: 2.4263
Epoch 3/50
279/279 [==============================] - 4s 13ms/step - loss: 0.0965 - val_loss: 1.9166
Epoch 4/50
279/279 [==============================] - 4s 13ms/step - loss: 0.1163 - val_loss: 2.8361
Epoch 5/50
279/279 [==============================] - 4s 13ms/step - loss: 0.0192 - val_loss: 2.7533
Epoch 6/50
279/279 [==============================] - 4s 13ms/step - loss: 0.0354 - val_loss: 1.2268
Epoch 7/50
279/279 [==============================] - 4s 13ms/step - loss: 0.0304 - val_loss: 1.7311
Epoch 8/50
279/279 [==============================] - 4s 13ms/step - loss: 0.0487 - val_loss: 0.3344
Epoch 9/50
279/279 [==============================] - 4s 13ms/step - loss: 0.0241 - val_loss: 0.8499
Epoch 10/50
279/279 [==============================] - 4s 13ms/step - loss: 0.1238 - val_lo

In [113]:
patience = 25
sequence_length = 60  # Number of days to look back
features = df.shape[1]  # Number of features (2: SPY and AMD)
epochs = 200
batch_size = 16
filters = 1000

early_stopping = EarlyStopping(patience=patience,monitor='val_loss',restore_best_weights=True)
checkpoint_filepath = "/kaggle/working Checkpoints/best_model{epoch:02d}_{val_loss:.2f}.keras"
model_checkpoint = ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_loss',verbose=1,save_best_only=True,mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=(patience/2),verbose=1,mode='auto',min_delta=0.0001,cooldown=0,min_lr=0.0)

# Normalize data
scaler = MinMaxScaler()
data = scaler.fit_transform(df)

# Prepare sequences and targets
X, y = [], []
for i in range(sequence_length, len(data)):
    X.append(data[i-sequence_length:i])
    y.append(data[i][0])  # SPY's price as target

X, y = np.array(X), np.array(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# TCN model architecture
def build_tcn(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters, kernel_size=2, dilation_rate=1, padding="causal")(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    # Stacking TCN blocks with residual connections
    for dilation_rate in [2, 4, 8, 16]:
        previous_x = x
        x = Conv1D(filters, kernel_size=2, dilation_rate=dilation_rate, padding="causal")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Add()([x, previous_x])  # Residual connection
    
    x = Flatten()(x)
    outputs = Dense(1)(x)
    
    model = Model(inputs, outputs)
    return model

# Compile model
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_tcn(input_shape)
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[early_stopping,reduce_lr])

# Evaluate model
mse = model.evaluate(X_test, y_test)
print("Test MSE:", mse)

predictions = model.predict(X_test)

Epoch 1/200
277/277 [==============================] - 6s 15ms/step - loss: 203.6101 - val_loss: 0.0109 - lr: 0.0010
Epoch 2/200
277/277 [==============================] - 3s 12ms/step - loss: 0.0556 - val_loss: 0.0409 - lr: 0.0010
Epoch 3/200
277/277 [==============================] - 3s 12ms/step - loss: 0.0683 - val_loss: 1.3876 - lr: 0.0010
Epoch 4/200
277/277 [==============================] - 3s 12ms/step - loss: 0.0363 - val_loss: 3.7748 - lr: 0.0010
Epoch 5/200
277/277 [==============================] - 3s 12ms/step - loss: 0.0963 - val_loss: 8.9279 - lr: 0.0010
Epoch 6/200
277/277 [==============================] - 3s 12ms/step - loss: 0.3644 - val_loss: 4.0055 - lr: 0.0010
Epoch 7/200
277/277 [==============================] - 3s 12ms/step - loss: 1.5069 - val_loss: 83.7538 - lr: 0.0010
Epoch 8/200
277/277 [==============================] - 3s 12ms/step - loss: 18.1707 - val_loss: 16645.4395 - lr: 0.0010
Epoch 9/200
277/277 [==============================] - 3s 12ms/step - lo

In [114]:
trace_0 = go.Scatter(y=history.history['loss'],
                     name='train',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=history.history['val_loss'],
                     name='test',
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]

layout = go.Layout(title="train/test logarithmic loss comparison",
                  template='plotly_dark',
                   height=600,
                   width=1000)

fig = go.Figure(data,layout=layout)
fig.show()

In [115]:
# predictions_df = pd.DataFrame(predictions)
# predictions_df = predictions_df.rename(columns={0: 'prediction'})

# y_test_df = pd.DataFrame(y_test)
# y_test_df = y_test_df.rename(columns={0: special_col})

# results_df = pd.concat([predictions_df,y_test_df],axis=1)

# results_df['difference'] = results_df['prediction'] - results_df['SPY']

# results_df

In [116]:
# Get the predictions from the model (shape: [n_samples, 1])
predictions = model.predict(X_test)
predictions_df = pd.DataFrame(predictions)
predictions_df = predictions_df.rename(columns={0: 'prediction'})
y_test_df = pd.DataFrame(y_test)
y_test_df = y_test_df.rename(columns={0: special_col})

# Since scaler was fitted on the entire dataframe, we need to match the shape
# Add dummy columns for AMD prices (same shape as predictions but filled with zeros or any value)
# The predictions correspond to SPY prices, and we'll use dummy data for the other columns (like AMD).
dummy_amds = np.zeros((predictions.shape[0], features - 1))  # Dummy data for the other features (e.g., AMD)

# Concatenate the predicted SPY prices with dummy data for AMD
predicted_data_with_dummy = np.concatenate([predictions, dummy_amds], axis=1)

# Inverse transform the concatenated data (this will reverse the scaling)
# scaler expects the same number of columns (features) as the original data
predicted_original_scale = scaler.inverse_transform(predicted_data_with_dummy)

# Extract only the first column (SPY prices) from the inverse transformed data
predicted_spy_prices = predicted_original_scale[:, 0]

# Now, predicted_spy_prices contains the predicted SPY prices in the original scale.
print(predicted_spy_prices)


predicted_spy_prices_df = pd.DataFrame(predicted_spy_prices)
predicted_spy_prices_df = predicted_spy_prices_df.rename(columns={0:'Predicted'})

y_test_dummys = np.zeros((y_test.shape[0], features-1))
actual_dummys = np.concatenate([y_test.reshape(-1,1), y_test_dummys],axis=1)
y_test_scaled = scaler.inverse_transform(actual_dummys)
actual_prices = y_test_scaled[:,0]
actual_prices_df = pd.DataFrame(actual_prices)
actual_prices_df = actual_prices_df.rename(columns={0: 'Actual'})

results_df = pd.concat([predicted_spy_prices_df,actual_prices_df],axis=1)
results_df['Differences'] = results_df['Predicted'] - results_df['Actual']
results_df = results_df.rename(columns={'Predicted': 'theta_hat','Actual': 'theta'})

print(f"Root Mean Squared Error: {rmse(results_df['theta_hat'],results_df['theta'])}")

results_df

35/35 [==============================] - 0s 9ms/step
[2992.11993953 3194.28409137 3223.67198817 ... 5700.56730743 5695.75464834
 5661.29727092]
Root Mean Squared Error: 234.18624588602648


,theta_hat,theta,Differences
0,2992.119940,3044.310059,-52.190119
1,3194.284091,3055.729980,138.554111
2,3223.671988,3080.820068,142.851920
3,3062.846979,3122.870117,-60.023138
4,3295.317308,3112.350098,182.967210
...,...,...,...
1103,5799.905131,5842.470215,-42.565084
1104,5965.219027,5841.470215,123.748813
1105,5700.567307,5864.669922,-164.102614
1106,5695.754648,5853.979980,-158.225332


In [107]:
trace_0 = go.Scatter(y=results_df['theta_hat'],
                     name='predicted',
                     mode='lines',
                     marker= {'color':'#FF6F61','opacity':0.9})

trace_1 = go.Scatter(y=results_df['theta'],
                     name=df.columns[0],
                     mode='lines',
                     marker= {'color':'#1d5dec','opacity':0.9})

data = [trace_0,trace_1]


layout = go.Layout(title=f"LSTM model predictions vs. actual price of {df.columns[0]}",
                  template='plotly_dark',
                   height=900,
                   width=1500)

fig = go.Figure(data,layout=layout)
fig.show()